<a href="https://colab.research.google.com/github/hemil19/Predict-The-Data-Scientists-Salary-In-India-Hackathon/blob/master/PredictDataScientistSalary.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline

In [4]:
train=pd.read_csv('/content/Final_Train_Dataset.csv')
test=pd.read_csv('/content/Final_Test_Dataset.csv')

In [5]:
train.drop(['Unnamed: 0'],axis=1,inplace=True)

In [6]:
train.head()

,experience,job_description,job_desig,job_type,key_skills,location,salary,company_name_encoded
0,5-7 yrs,Exp: Minimum 5 years;Good understanding of IOC...,Senior Exploit and Vulnerability Researcher,NaN,"team skills, communication skills, analytical ...",Delhi NCR(Vikas Puri),6to10,3687
1,10-17 yrs,He should have handled a team of atleast 5-6 d...,Head SCM,NaN,"ppc, logistics, inventory management, supply c...",Sonepat,10to15,458
2,5-9 yrs,Must be an effective communicator (written & s...,Deputy Manager - Talent Management & Leadershi...,Analytics,"HR Analytics, Employee Engagement, Training, S...",Delhi NCR,15to25,4195
3,7-10 yrs,7 - 10 years of overall experience in data e...,Associate Manager Data Engineering,Analytics,"SQL, Javascript, Automation, Python, Ruby, Ana...",Bengaluru,10to15,313
4,1-3 yrs,Chartered Accountancy degree or MBA in Finance...,TS- GSA- Senior Analyst,NaN,"accounting, finance, cash flow, financial plan...",Gurgaon,3to6,1305


In [7]:
job_map={'Analytics':'Analytics', 'analytics':'Analytics', 'Analytic':'Analytics', 'ANALYTICS':'Analytics', 'analytic':'Analytics',np.nan:'Other'}

In [8]:
train.isnull().sum()

experience                  0
job_description          4418
job_desig                   0
job_type                15005
key_skills                  1
location                    0
salary                      0
company_name_encoded        0
dtype: int64

In [9]:
test.isnull().sum()

experience                 0
job_description         1522
job_desig                  0
job_type                4964
key_skills                 0
location                   0
company_name_encoded       0
dtype: int64

In [10]:
combine=train.append(test)
combine.reset_index(inplace=True)
combine.drop('index',axis=1,inplace=True)
combine.shape

(26403, 8)

In [11]:
combine['job_type']=combine['job_type'].map(job_map)

In [12]:
combine.head()

,experience,job_description,job_desig,job_type,key_skills,location,salary,company_name_encoded
0,5-7 yrs,Exp: Minimum 5 years;Good understanding of IOC...,Senior Exploit and Vulnerability Researcher,Other,"team skills, communication skills, analytical ...",Delhi NCR(Vikas Puri),6to10,3687
1,10-17 yrs,He should have handled a team of atleast 5-6 d...,Head SCM,Other,"ppc, logistics, inventory management, supply c...",Sonepat,10to15,458
2,5-9 yrs,Must be an effective communicator (written & s...,Deputy Manager - Talent Management & Leadershi...,Analytics,"HR Analytics, Employee Engagement, Training, S...",Delhi NCR,15to25,4195
3,7-10 yrs,7 - 10 years of overall experience in data e...,Associate Manager Data Engineering,Analytics,"SQL, Javascript, Automation, Python, Ruby, Ana...",Bengaluru,10to15,313
4,1-3 yrs,Chartered Accountancy degree or MBA in Finance...,TS- GSA- Senior Analyst,Other,"accounting, finance, cash flow, financial plan...",Gurgaon,3to6,1305


In [13]:
combine.shape

(26403, 8)

In [14]:
combine['job_desig']=combine['job_desig'].str.lower()

In [15]:
combine['key_skills']=combine['key_skills'].str.lower()

In [16]:
keys=combine.groupby('salary').agg({'key_skills':pd.Series.mode}).index

In [17]:
values=combine.groupby('salary').agg({'key_skills':pd.Series.mode}).values

In [18]:
dict={}
for i in range(0,len(keys)):
  dict[keys[i]]=values[i]

In [19]:
combine.loc[846,'key_skills']=dict[combine.loc[846,'salary']]

In [20]:
combine.drop('job_description',axis=1,inplace=True)

In [21]:
combine.isnull().sum()

experience                 0
job_desig                  0
job_type                   0
key_skills                 0
location                   0
salary                  6601
company_name_encoded       0
dtype: int64

In [22]:
combine['total_skills']=[len(i) for i in combine['key_skills']]

In [23]:
combine.head()

,experience,job_desig,job_type,key_skills,location,salary,company_name_encoded,total_skills
0,5-7 yrs,senior exploit and vulnerability researcher,Other,"team skills, communication skills, analytical ...",Delhi NCR(Vikas Puri),6to10,3687,72
1,10-17 yrs,head scm,Other,"ppc, logistics, inventory management, supply c...",Sonepat,10to15,458,78
2,5-9 yrs,deputy manager - talent management & leadershi...,Analytics,"hr analytics, employee engagement, training, s...",Delhi NCR,15to25,4195,67
3,7-10 yrs,associate manager data engineering,Analytics,"sql, javascript, automation, python, ruby, ana...",Bengaluru,10to15,313,73
4,1-3 yrs,ts- gsa- senior analyst,Other,"accounting, finance, cash flow, financial plan...",Gurgaon,3to6,1305,72


In [24]:
combine.dtypes

experience              object
job_desig               object
job_type                object
key_skills              object
location                object
salary                  object
company_name_encoded     int64
total_skills             int64
dtype: object

In [25]:
import re
combine['location']=[re.sub(r" ?\([^)]+\)", "",i) for i in combine['location']]

In [26]:
value=[]
for i in combine['location'].str.split(','):
  if(len(i)>1):
    value.append('multiple')
  else:
    value.append(i[0])

In [27]:
combine['location']=value

In [28]:
combine.head()

,experience,job_desig,job_type,key_skills,location,salary,company_name_encoded,total_skills
0,5-7 yrs,senior exploit and vulnerability researcher,Other,"team skills, communication skills, analytical ...",Delhi NCR,6to10,3687,72
1,10-17 yrs,head scm,Other,"ppc, logistics, inventory management, supply c...",Sonepat,10to15,458,78
2,5-9 yrs,deputy manager - talent management & leadershi...,Analytics,"hr analytics, employee engagement, training, s...",Delhi NCR,15to25,4195,67
3,7-10 yrs,associate manager data engineering,Analytics,"sql, javascript, automation, python, ruby, ana...",Bengaluru,10to15,313,73
4,1-3 yrs,ts- gsa- senior analyst,Other,"accounting, finance, cash flow, financial plan...",Gurgaon,3to6,1305,72


In [29]:
combine.shape

(26403, 8)

In [30]:
len(combine['key_skills'].unique())

15215

In [31]:
combine.isnull().sum()

experience                 0
job_desig                  0
job_type                   0
key_skills                 0
location                   0
salary                  6601
company_name_encoded       0
total_skills               0
dtype: int64

In [32]:
X = combine[combine['salary'].isnull()!=True].drop(['salary'], axis=1)
y = combine[combine['salary'].isnull()!=True]['salary']

test = combine[combine['salary'].isnull()==True].drop(['salary'], axis=1)

X.shape, y.shape, test.shape

((19802, 7), (19802,), (6601, 7))

In [33]:
X.dtypes

experience              object
job_desig               object
job_type                object
key_skills              object
location                object
company_name_encoded     int64
total_skills             int64
dtype: object

In [34]:
X.shape,test.shape

((19802, 7), (6601, 7))

In [35]:
salary_map={'6to10':2, '10to15':3, '15to25':4, '3to6':1, '25to50':5, '0to3':0}
y=y.map(salary_map)

In [36]:
from sklearn.metrics import accuracy_score

In [37]:
submission=pd.read_excel('/content/sample_submission.xlsx')
submission.head()

,salary
0,6to10
1,10to15
2,15to25
3,10to15
4,3to6


In [38]:
from sklearn.model_selection import train_test_split
X_train,X_test,y_train,y_test=train_test_split(X,y,test_size=0.2,random_state=42)

In [41]:
!pip install catboost

     |████████████████████████████████| 64.8MB 60kB/s 


In [42]:
from catboost import CatBoostClassifier
m = CatBoostClassifier(n_estimators=5000,
                           random_state=1994,
                           eval_metric='Accuracy',
                           learning_rate=0.03, 
                           max_depth=5,
                           cat_features=[0,1,2,3,4])
m.fit(X_train, y_train,
          eval_set=[(X_train,y_train),(X_test, y_test)],
          early_stopping_rounds=200,
          verbose=200)
accuracy_score(m.predict(X_test),y_test)

0:	learn: 0.3901900	test: 0.3905056	test1: 0.3766726	best: 0.3766726 (0)	total: 175ms	remaining: 14m 35s
200:	learn: 0.4918250	test: 0.3148160	test1: 0.5054279	best: 0.5056804 (199)	total: 19.6s	remaining: 7m 48s
400:	learn: 0.5156240	test: 0.2807272	test1: 0.5258773	best: 0.5263822 (370)	total: 42.1s	remaining: 8m 2s
600:	learn: 0.5263557	test: 0.2790859	test1: 0.5339561	best: 0.5357233 (584)	total: 1m 7s	remaining: 8m 11s
800:	learn: 0.5365192	test: 0.2794647	test1: 0.5385004	best: 0.5392578 (759)	total: 1m 31s	remaining: 7m 58s
1000:	learn: 0.5439051	test: 0.2793384	test1: 0.5402676	best: 0.5410250 (953)	total: 1m 55s	remaining: 7m 40s
1200:	learn: 0.5539423	test: 0.2794647	test1: 0.5432971	best: 0.5438021 (1148)	total: 2m 20s	remaining: 7m 23s
1400:	learn: 0.5630958	test: 0.2778865	test1: 0.5453168	best: 0.5455693 (1364)	total: 2m 44s	remaining: 7m 3s
1600:	learn: 0.5694716	test: 0.2780759	test1: 0.5455693	best: 0.5473365 (1462)	total: 3m 9s	remaining: 6m 41s
Stopped by overfitting

0.5473365311789952

In [48]:
from catboost import CatBoostClassifier,Pool, cv
errCB = []
y_pred_tot_cb = []
from sklearn.model_selection import KFold,StratifiedKFold

fold = StratifiedKFold(n_splits=5)
i = 1
for train_index, test_index in fold.split(X,y):
    x_train, x_val = X.iloc[train_index], X.iloc[test_index]
    y_train, y_val = y[train_index], y[test_index]
    m = CatBoostClassifier(n_estimators=1200,
                           random_state=1994,
                           eval_metric='Accuracy',
                           learning_rate=0.03, 
                           max_depth=5,
                           cat_features=[0,1,2,3,4])
    m.fit(x_train, y_train,
          eval_set=[(x_train,y_train),(x_val, y_val)],
          early_stopping_rounds=200,
          verbose=200)
    pred_y = m.predict(x_val)
    print("err_cb: ",accuracy_score(y_val,pred_y))
    errCB.append(accuracy_score(y_val,pred_y))
    pred_test = m.predict(test)
    i = i + 1
    y_pred_tot_cb.append(pred_test)

0:	learn: 0.3727669	test: 0.3829304	test1: 0.3726332	best: 0.3726332 (0)	total: 99.3ms	remaining: 1m 59s
200:	learn: 0.4853229	test: 0.3331860	test1: 0.5152739	best: 0.5175461 (182)	total: 19.8s	remaining: 1m 38s
400:	learn: 0.5056499	test: 0.3166467	test1: 0.5306741	best: 0.5306741 (400)	total: 42.8s	remaining: 1m 25s
600:	learn: 0.5187804	test: 0.3018118	test1: 0.5379955	best: 0.5379955 (600)	total: 1m 7s	remaining: 1m 7s
800:	learn: 0.5318477	test: 0.2948046	test1: 0.5407725	best: 0.5410250 (797)	total: 1m 32s	remaining: 46.1s
1000:	learn: 0.5422006	test: 0.2951203	test1: 0.5405201	best: 0.5420348 (829)	total: 1m 57s	remaining: 23.3s
Stopped by overfitting detector  (200 iterations wait)

bestTest = 0.5420348397
bestIteration = 829

Shrink model to first 830 iterations.
err_cb:  0.5420348396869478
0:	learn: 0.3636134	test: 0.3692949	test1: 0.3643019	best: 0.3643019 (0)	total: 102ms	remaining: 2m 2s
200:	learn: 0.4833028	test: 0.3339436	test1: 0.5142641	best: 0.5152739 (192)	total: 1

In [49]:
errCB

[0.5420348396869478,
 0.5539005301691492,
 0.5613636363636364,
 0.5409090909090909,
 0.5636363636363636]

In [50]:
#submission['salary']=y_pred_tot_cb[-1]
submission['salary']=np.mean(y_pred_tot_cb,0)
#submission['salary']=m.predict(test)

In [51]:
submission['salary'].unique()

array([4. , 0. , 3. , 2. , 5. , 3.2, 3.8, 3.6, 4.8, 1.6, 1. , 4.4, 4.6,
       1.8, 0.8, 2.8, 2.4, 2.6, 0.6, 4.2, 3.4, 2.2, 1.4, 0.2, 1.2, 0.4])

In [52]:
dict_ans={4. :4, 0.:0, 3.:3, 2.:2, 5.:5, 3.2:3, 3.8:4, 3.6:4, 4.8:5, 1.6:2, 1.:1 , 4.4:4, 4.6:5,
       1.8:2, 0.8:1, 2.8:3, 2.4:2, 2.6:3, 0.6:1, 4.2:4, 3.4:3, 2.2:2, 1.4:1, 0.2:0, 1.2:1, 0.4:0}

In [53]:
submission['salary']=submission['salary'].map(dict_ans)

In [54]:
submission['salary']=submission['salary'].map({2:'6to10',3:'10to15', 4:'15to25',1:'3to6',5:'25to50',0:'0to3'})

In [55]:
submission['salary']

0       15to25
1         0to3
2       10to15
3         0to3
4         0to3
         ...  
6596     6to10
6597      3to6
6598    15to25
6599    15to25
6600     6to10
Name: salary, Length: 6601, dtype: object

In [56]:
submission.to_csv('predsal_01.csv',index=False)